# Try the New MCP Agent - connected to a Test Postgres DB

### Features include:

- 🔍 Database Health - analyze index health, connection utilization, buffer cache, vacuum health, sequence limits, replication lag, and more.
- ⚡ Index Tuning - explore thousands of possible indexes to find the best solution for your workload, using industrial-strength algorithms.
- 📈 Query Plans - validate and optimize performance by reviewing EXPLAIN plans and simulating the impact of hypothetical indexes.
- 🧠 Schema Intelligence - context-aware SQL generation based on detailed understanding of the database schema.
- 🛡️ Safe SQL Execution - configurable access control, including support for read-only mode and safe SQL parsing, making it usable for both development and production.

In [ ]:
%pip install -q langchain-mcp-adapters langgraph langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install -q psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.4 MB/s eta 0:00:00


In [ ]:
db_url = "your_hosted_postgres_db_connection_url"

In [ ]:
# Don't run this as data is already loaded in the DB
# this code is used to load the sample data to the db

# import psycopg2
# import requests

# # 1. Download the SQL script from GitHub
# sql_url = 'https://raw.githubusercontent.com/pthom/northwind_psql/master/northwind.sql'
# try:
#     response = requests.get(sql_url)
#     response.raise_for_status()  # Raise an exception for bad status codes
#     sql_script = response.text
#     print("SQL script downloaded successfully.")
# except requests.exceptions.RequestException as e:
#     print(f"Error downloading script: {e}")
#     sql_script = None

# # 2. Connect to your Render PostgreSQL database and execute the script
# if sql_script:
#     # Replace with your actual database credentials from Render

#     try:
#         # Connect to the database
#         conn = psycopg2.connect(db_url)
#         conn.autocommit = True  # Set autocommit to true to run the script commands immediately
#         cursor = conn.cursor()
#         print("Database connection successful. Executing script...")

#         # Execute the entire SQL script
#         cursor.execute(sql_script)

#         print("Northwind database script executed successfully.")

#     except Exception as e:
#         print(f"An error occurred: {e}")

#     finally:
#         # Clean up the connection
#         if 'cursor' in locals():
#             cursor.close()
#         if 'conn' in locals():
#             conn.close()
#         print("Database connection closed.")



In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

try:
    # Create a database engine
    engine = create_engine(db_url)
    print("Successfully created database engine.")

    # Connect to the database
    with engine.connect() as connection:
        print("Database connection successful. Running verification queries...")

        # --- Verification Query 1: List all tables ---
        # This confirms that the tables were created by the script.
        print("\n--- Verifying table creation ---")
        list_tables_query = text("""
            SELECT tablename
            FROM pg_catalog.pg_tables
            WHERE schemaname != 'pg_catalog' AND
                  schemaname != 'information_schema';
        """)
        tables_df = pd.read_sql(list_tables_query, connection)
        print("Tables found in the database:")
        print(tables_df)

        # --- Verification Query 2: Count rows in the 'customers' table ---
        # This confirms that data was inserted.
        print("\n--- Verifying row count in 'customers' table ---")
        count_query = text("SELECT COUNT(*) FROM customers;")
        row_count = connection.execute(count_query).scalar()
        print(f"Number of rows in 'customers' table: {row_count}")
        # The Northwind script should insert 91 customers.

        # --- Verification Query 3: Select a sample of data from the 'products' table ---
        # This allows for a visual inspection of the data.
        print("\n--- Displaying sample data from 'products' table ---")
        sample_data_query = text("SELECT * FROM products LIMIT 5;")
        products_df = pd.read_sql(sample_data_query, connection)
        print("Sample of data from the 'products' table:")
        print(products_df)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if 'engine' in locals():
        engine.dispose()
        print("\nDatabase connection closed.")

Successfully created database engine.
Database connection successful. Running verification queries...

--- Verifying table creation ---
Tables found in the database:
                 tablename
0                us_states
1                customers
2                   orders
3                employees
4                 shippers
5                 products
6               categories
7                suppliers
8                   region
9              territories
10    employee_territories
11   customer_demographics
12  customer_customer_demo
13           order_details

--- Verifying row count in 'customers' table ---
Number of rows in 'customers' table: 91

--- Displaying sample data from 'products' table ---
Sample of data from the 'products' table:
   product_id                  product_name  supplier_id  category_id  \
0           1                          Chai            8            1   
1           2                         Chang            1            1   
2           3           

In [ ]:
!python --version

Python 3.11.13


In [ ]:
import json

In [ ]:
# Step 3: Configure the LangChain client to connect to the local MCP server
from langchain_mcp_adapters.client import MultiServerMCPClient

# The server started by npx runs on localhost:8000 within the Colab environment
client = MultiServerMCPClient(
    {
        "postgres": {
          "url": "your_hosted_mcp_server_url",
            "transport": "sse",
        }
    }
)

print("MCP Client configured. You can now get tools and create your agent.")

MCP Client configured. You can now get tools and create your agent.


In [ ]:
# Asynchronously get the tools from the client
tools = await client.get_tools()

from pprint import pprint
pprint(tools)

[StructuredTool(name='list_schemas', description='List all schemas in the database', args_schema={'properties': {}, 'title': 'list_schemasArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7b8f911a71a0>),
 StructuredTool(name='list_objects', description='List objects in a schema', args_schema={'properties': {'schema_name': {'description': 'Schema name', 'title': 'Schema Name', 'type': 'string'}, 'object_type': {'default': 'table', 'description': "Object type: 'table', 'view', 'sequence', or 'extension'", 'title': 'Object Type', 'type': 'string'}}, 'required': ['schema_name'], 'title': 'list_objectsArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7b8f911a7e20>),
 StructuredTool(name='get_object_details', description='Show detailed information about a database object', args_schema={'properti

In [ ]:
from google.colab import userdata
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=userdata.get("AZURE_OPENAI_API_KEY"),
    azure_endpoint=userdata.get("AZURE_OPENAI_ENDPOINT"),
    openai_api_version=userdata.get("AZURE_OPENAI_VERSION"),
    azure_deployment="gpt-4.1",
    temperature=0,
    streaming=True,
)

In [ ]:
async def main():
    tools = await client.get_tools()
    agent = create_react_agent(llm, tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "user-convo-123"}}

    # First question
    print("\n--- Turn 1 ---")
    q1 = "How many customers are in the customers table?"
    async for event in agent.astream_events({"messages": [{"role": "user", "content": q1}]}, config=config, version="v1"):
        if event["event"] == "on_chat_model_stream" and event["data"]["chunk"]["content"]:
            print(event["data"]["chunk"]["content"], end="", flush=True)

    # Follow-up, demonstrating memory
    print("\n\n--- Turn 2 ---")
    q2 = "And how many orders are there?"
    async for event in agent.astream_events({"messages": [{"role": "user", "content": q2}]}, config=config, version="v1"):
        if event["event"] == "on_chat_model_stream" and event["data"]["chunk"]["content"]:
            print(event["data"]["chunk"]["content"], end="", flush=True)

await main()